<a href="https://colab.research.google.com/github/PritamKhaire/PritamKhaire/blob/main/Language_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import pandas as pd
import numpy as np
import re
import os
import time
import string
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import unicodedata

from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [6]:
# Fetch the text data
lines = pd.read_table('./sample_data/mar.txt', names = ['eng', 'mar'])

In [7]:
lines["english"] = lines.index
lines["marathi"] = lines.eng

In [8]:
lines.drop(["eng", "mar"], axis = 1, inplace = True)
lines = lines.reset_index(drop = True)

In [9]:
lines.describe()

,english,marathi
count,41028,41028
unique,26581,40085
top,He works in a bank.,तो माझा भाऊ आहे.
freq,12,4


In [10]:
lines.head()

,english,marathi
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [11]:
def unicode_to_ascii(input_string):
  return "".join(c for c in unicodedata.normalize('NFD', input_string) if unicodedata.category(c) != 'Mn')

In [12]:
def preprocess_sentence(sentence):
  w = unicode_to_ascii(sentence.lower().strip())

  # Create a space between a word and punctuation following
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+ ', " ", w )

  w = re.sub(r"'", r'', w)
  

  exclude = set(string.punctuation)
  remove_digits = str.maketrans('', '', string.digits)
  w = "".join(ch for ch in w if ch not in exclude)
  w = w.translate(remove_digits)
  w = re.sub(r"[२३०८१५७९४६]", "", w)
  w = re.sub(r" +", r" ", w)
  w = w.strip()
  w = '<statr> ' + w + ' <end>'
  return w

In [13]:
def create_dataset(file_name):
  file_name.english = file_name.english.apply(lambda x : preprocess_sentence(x))
  file_name.marathi = file_name.marathi.apply(lambda x : preprocess_sentence(x))
  return file_name.marathi, file_name.english

In [ ]:
# lines = create_dataset(lines)

In [14]:
lines.head()

,english,marathi
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [15]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding="post")
  return tensor, lang_tokenizer

In [16]:
def load_dataset(lines):
  target_lang, input_lang = create_dataset(lines)
  input_tensor, inp_lang_tokenizer = tokenize(input_lang)
  target_tensor, targ_lang_tokenizer = tokenize(target_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [17]:
input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer = load_dataset(lines)

In [18]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [19]:
print(input_tensor.shape)
print(target_tensor.shape)

(41028, 36)
(41028, 37)


In [20]:
max_len_targ, max_len_inp = target_tensor.shape[1], input_tensor.shape[1]

In [21]:
## Creating training and testing set using 80-20
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

32822 32822 8206 8206


In [22]:
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      print(f'{t} ------> {lang.index_word[t]}')

In [23]:
print("Input language: Index to word mapping...")
convert(inp_lang_tokenizer, input_tensor_train[0])
print("Target Language; index to word mapping")
convert(targ_lang_tokenizer, target_tensor_train[0])

Input language: Index to word mapping...
1 ------> <statr>
30 ------> im
560 ------> standing
10 ------> in
6 ------> the
2375 ------> shade
2 ------> <end>
Target Language; index to word mapping
1 ------> <statr>
4 ------> मी
3746 ------> सावलीत
603 ------> उभा
3 ------> आह
2 ------> <end>


In [24]:
# Model parameter
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
EPOCH_SIZE = len(input_tensor_train) // BATCH_SIZE
EMBEDING_DIM = 256
UNITS = 1024

VOCAB_INPUT_SIZE = len(inp_lang_tokenizer.word_index) + 1
VOCAB_TAR_SIZE = len(targ_lang_tokenizer.word_index) + 1

In [25]:
# Create tensor dataset
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

In [26]:
example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch.shape)
print(example_target_batch.shape)

(64, 36)
(64, 37)


In [27]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [28]:
encoder = Encoder(VOCAB_INPUT_SIZE, EMBEDING_DIM, UNITS, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)

Encoder output shape: (batch size, sequence length, units) (64, 36, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [29]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [30]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units)", attention_result.shape)
print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 36, 1)


In [31]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [32]:
decoder = Decoder(VOCAB_TAR_SIZE, EMBEDING_DIM, UNITS, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)

Decoder output shape: (batch_size, vocab size) (64, 12574)


In [33]:
# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction = 'none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [34]:
# Checkpoint
checkpoint_dir = './sample_data'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder = encoder,
                                 decoder = decoder)

In [ ]:
# pass input thru encode which returns with encoder output and hidden state
# pass encoder output, hidden state and decoder input to decoder function
# prediction output of decoder return decoder output and hidden state
# Decoder output is passed back to model and hidden state is used to calculate the loss of the model

In [35]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<statr>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [36]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(EPOCH_SIZE)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix=checkpoint_prefix)

Epoch 1 Batch 0 Loss 1.5308
Epoch 1 Batch 100 Loss 0.8139
Epoch 1 Batch 200 Loss 0.8197
Epoch 1 Batch 300 Loss 0.9013
Epoch 1 Batch 400 Loss 0.8243
Epoch 1 Batch 500 Loss 0.7928
Epoch 2 Batch 0 Loss 0.7156
Epoch 2 Batch 100 Loss 0.6660
Epoch 2 Batch 200 Loss 0.6732
Epoch 2 Batch 300 Loss 0.6640
Epoch 2 Batch 400 Loss 0.6743
Epoch 2 Batch 500 Loss 0.6240
Epoch 3 Batch 0 Loss 0.5358
Epoch 3 Batch 100 Loss 0.5238
Epoch 3 Batch 200 Loss 0.5318
Epoch 3 Batch 300 Loss 0.5148
Epoch 3 Batch 400 Loss 0.5251
Epoch 3 Batch 500 Loss 0.4426
Epoch 4 Batch 0 Loss 0.4122
Epoch 4 Batch 100 Loss 0.4983
Epoch 4 Batch 200 Loss 0.3835
Epoch 4 Batch 300 Loss 0.3527
Epoch 4 Batch 400 Loss 0.2945
Epoch 4 Batch 500 Loss 0.3331
Epoch 5 Batch 0 Loss 0.2773
Epoch 5 Batch 100 Loss 0.3257
Epoch 5 Batch 200 Loss 0.2525
Epoch 5 Batch 300 Loss 0.2311
Epoch 5 Batch 400 Loss 0.2512
Epoch 5 Batch 500 Loss 0.2315
Epoch 6 Batch 0 Loss 0.1342
Epoch 6 Batch 100 Loss 0.1454
Epoch 6 Batch 200 Loss 0.1341
Epoch 6 Batch 300 Loss

In [37]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, UNITS))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<statr>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang_tokenizer.index_word[predicted_id] + ' '

    if targ_lang_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [38]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input:', sentence)
  print('Predicted translation:', result)

  # attention_plot = attention_plot[:len(result.split(' ')),
  #                                 :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [40]:
translate("what is your name")

Input: <statr> what is your name <end>
Predicted translation: तमच नाव काय आह <end> 


In [48]:
import nltk.translate.bleu_score as bleu
reference = "what is your name"
result =  translate(reference)
print(bleu.sentence_bleu(reference, result))
# print('BLEU score: {}'.format( bleu.sentence_bleu(reference, result)) )

Input: <statr> what is your name <end>
Predicted translation: तमच नाव काय आह <end> 


TypeError: ignored

In [41]:
!zip -r /content/sample_data.zip /content/sample_data

  adding: content/sample_data/ (stored 0%)
  adding: content/sample_data/README.md (deflated 42%)
  adding: content/sample_data/anscombe.json (deflated 83%)
  adding: content/sample_data/ckpt-10.index (deflated 69%)
  adding: content/sample_data/ckpt-22.index (deflated 69%)
  adding: content/sample_data/ckpt-1.data-00000-of-00001 (deflated 14%)
  adding: content/sample_data/ckpt-14.data-00000-of-00001 (deflated 13%)
  adding: content/sample_data/ckpt-3.data-00000-of-00001 (deflated 14%)
  adding: content/sample_data/ckpt-5.index (deflated 69%)
  adding: content/sample_data/ckpt-18.data-00000-of-00001 (deflated 13%)
  adding: content/sample_data/ckpt-5.data-00000-of-00001 (deflated 14%)
  adding: content/sample_data/ckpt-3.index (deflated 69%)
  adding: content/sample_data/ckpt-17.data-00000-of-00001 (deflated 13%)
  adding: content/sample_data/ckpt-7.index (deflated 69%)
  adding: content/sample_data/ckpt-18.index (deflated 69%)
  adding: content/sample_data/ckpt-11.data-00000-of-00001